In [1]:
%load_ext autoreload
%autoreload 2

In [59]:
import scanpy as sc
import pandas as pd
import newick
from Bio import Phylo
from anndata import AnnData
import ete3
from scipy.sparse import issparse
import os
import sys
import pickle as pkl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#import utils
from jax import numpy as jnp
import ot
import lineageot
from collections import defaultdict
import networkx as nx

In [60]:
from moscot.framework.estimators import LineageEstimator

In [3]:
import sys
sys.path.append("../..")
from moscot.framework import zoe_utils

In [4]:
data_dir = "/Users/DominikKlein/Documents/PhD/data/tedsim_zoe/adatas/adata_pa0.8_ss0.6.h5ad"

In [5]:
adata = sc.read(data_dir)

Only considering the two last: ['.6', '.h5ad'].
Only considering the two last: ['.6', '.h5ad'].


In [6]:
tree = newick.loads(adata.uns['tree'])[0]

In [7]:
adatas = zoe_utils.create_adatas(adata, 6)

In [67]:
top_leaves = list(adatas['top'].obs_names.to_numpy().astype(int) - 1)
bottom_leaves = list(adatas['bottom'].obs_names.to_numpy().astype(int) - 1)
G = zoe_utils.gt_tree(adatas['processed'], tree, create_cell_bool=False)



In [63]:
G = zoe_utils.gt_tree(adatas['processed'], tree, create_cell_bool=False)


In [71]:
dep = 6

In [68]:
top, bottom = zoe_utils.cut_tree_at_depth(G, d)

KeyError: 'node_depth'

In [73]:
tree = newick.loads(adatas['orig'].uns['tree'])[0]
G = zoe_utils.gt_tree(adatas['processed'], tree, create_cell_bool=False)
d = zoe_utils.add_node_depths(adatas['processed'], G)

top, bottom = zoe_utils.cut_tree_at_depth(G, dep)

In [75]:
top, bottom

(<networkx.classes.digraph.DiGraph at 0x7facd378e6d0>,
 <networkx.classes.digraph.DiGraph at 0x7faccf393370>)

In [80]:
import lineageot.inference as sim_inf
bottom_leaves = sim_inf.get_leaves(bottom, include_root=False)
top_leaves = sim_inf.get_leaves(top, include_root=False)
top_cell_ids = [top.nodes[n]['cellID'] for n in top_leaves]
#assert all(top.nodes[n]['node_depth'] == d for n in top_leaves)
adata_tmp = adata.copy()
zoe_utils.process(adata)
adata_bottom = adata_tmp[:1024].copy()
adata_top = adata_tmp[adata_tmp.obs['cellID'].isin(top_cell_ids)][[str(c) for c in top_cell_ids]].copy()
jdata = adata_top.concatenate(adata_bottom, batch_key='time_point')

In [82]:
zoe_utils.process(adata_bottom)
zoe_utils.process(adata_top)
zoe_utils.process(jdata)

In [84]:
jdata.obs.time_point.unique()

['0', '1']
Categories (2, object): ['0', '1']

In [91]:
jdata.obs.time_point = jdata.obs.time_point.astype(int, copy=False)

In [92]:
jdata.obs.time_point.dtype

dtype('int64')

In [93]:
key = "time_point"


In [94]:
estimator = LineageEstimator(jdata, {0:top, 1:bottom})

In [ ]:
estimator.prepare()